In [14]:
import numpy as np
import xarray as xr
import napari
from mpl_toolkits.basemap import Basemap
from tqdm.notebook import tqdm
import OpenVisus as ov
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import plotly.graph_objects as go
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
from matplotlib.widgets import Slider, RadioButtons
import matplotlib.colors as mcolors
import matplotlib
import logging
matplotlib.use('Agg')

os.environ['VISUS_CACHE']= "./visus_can_be_deleted"

In [2]:
# !pip install napari
# %matplotlib inline
# %matplotlib notebook

In [3]:
# %%time
# variables = ['salt', 'u', 'v', 'w', 'theta']
# data_list = []

# for variable in tqdm(variables, desc="Processing variables"):
#     # DONOT TOUCH
#     base_url = "https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/"
#     end_url = "?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco"
#     if variable in ["theta", "w"]:
#         base_dir = f"mit_output/llc2160_{variable}/llc2160_{variable}.idx"
#     elif variable == "u":
#         base_dir = "mit_output/llc2160_arco/visus.idx"
#     else:
#         base_dir = f"mit_output/llc2160_{variable}/{variable}_llc2160_x_y_depth.idx"

#     field = base_url + base_dir + end_url
#     db = ov.LoadDataset(field)

#     z = db.getLogicBox()[1][2]
#     temp_data = np.expand_dims(db.read(time=0, quality=-15, z=(0, z)), axis=0)

#     for i in tqdm(range(1, len(db.getTimesteps()) - 1, 25), desc=f"Processing {variable}", leave=False):
#         local_temp_data = np.expand_dims(db.read(time=i, quality=-15, z=(0, z)), axis=0)
#         temp_data = np.concatenate((temp_data, local_temp_data), axis=0)

#     data_list.append(temp_data)

# data = np.stack(data_list, axis=0)
# data = np.flip(data, axis=3)
# print(data.shape)

In [4]:
# %%time

# var_index = 0
# time_index = 0
# depth_index = 0

# fig, ax = plt.subplots(figsize=(9, 7))
# plt.subplots_adjust(left=0.25, bottom=0.3)

# vmin = 0
# vmax = 50
# vcenter = 30
# norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
# cmap = 'plasma'

# img = ax.imshow(
#     data[var_index, time_index, depth_index, :, :],
#     cmap=cmap,
#     norm=norm
# )
# ax.set_title(f'Variable {var_index}, Time {time_index}, Depth {depth_index}')
# plt.colorbar(img, ax=ax)

# ax_var = plt.axes([0.05, 0.5, 0.15, 0.35], facecolor='lightgoldenrodyellow')
# var_selector = RadioButtons(ax_var, [f'Var {i}' for i in range(data.shape[0])], active=0)

# ax_time = plt.axes([0.25, 0.18, 0.65, 0.03], facecolor='lightgoldenrodyellow')
# time_slider = Slider(ax_time, 'Time', 0, data.shape[1]-1, valinit=0, valstep=1)

# ax_depth = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor='lightgoldenrodyellow')
# depth_slider = Slider(ax_depth, 'Depth', 0, data.shape[2]-1, valinit=0, valstep=1)

# def update(val):
#     var_idx = int(var_selector.value_selected.split()[1])
#     time_idx = int(time_slider.val)
#     depth_idx = int(depth_slider.val)
#     img.set_data(data[var_idx, time_idx, depth_idx])
#     ax.set_title(f'Variable {var_idx}, Time {time_idx}, Depth {depth_idx}')
#     fig.canvas.draw_idle()

# var_selector.on_clicked(update)
# time_slider.on_changed(update)
# depth_slider.on_changed(update)

# plt.show()

In [8]:
def downsample_latlon(latitudes, longitudes, target_shape):
    factor_lat = latitudes.shape[0] // target_shape[0]
    factor_lon = latitudes.shape[1] // target_shape[1]
    latitudes_downsampled = latitudes[::factor_lat, ::factor_lon]
    longitudes_downsampled = longitudes[::factor_lat, ::factor_lon]
    return latitudes_downsampled, longitudes_downsampled

In [10]:
temp = xr.open_dataset('geos_c1440_lats_lons_2D.nc')

latitudes = temp['lats']
longitudes = temp['lons']

lat_shape = latitudes.shape
lon_shape = longitudes.shape

# Determine the total number of rows and columns
total_rows, total_cols = latitudes.shape

# Assuming the grid can be divided into 6 faces (2 rows of 3 faces each)
faces_per_row = 3
faces_per_col = 2
face_rows = total_rows // faces_per_col
face_cols = total_cols // faces_per_row

lat_faces = []
lon_faces = []

for row in range(faces_per_col):
    for col in range(faces_per_row):
        start_row = row * face_rows
        end_row = start_row + face_rows
        start_col = col * face_cols
        end_col = start_col + face_cols

        lat_face = latitudes[start_row:end_row, start_col:end_col]
        lon_face = longitudes[start_row:end_row, start_col:end_col]

        lat_faces.append(lat_face)
        lon_faces.append(lon_face)


In [ ]:
variables = ['T', 'QI', 'QL', 'FCLD']
variable = variables[1]

data_file=[f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_0_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
          f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_1_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
          f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_2_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
          f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_3_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
          f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_4_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
          f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_5_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",]
total_faces = []
for actual_file_path in tqdm(data_file, desc="Loading faces"):
    db = ov.LoadDataset(actual_file_path)
    data=[]
    for i in tqdm(range(0, len(db.getTimesteps()), 24), desc="Weekly timesteps", leave=False):
        data.append(db.read(time=i,quality=-3,z=[24,25]))
    total_faces.append(data)

In [ ]:
data_faces = []
for i in range(len(total_faces)):
    data_faces.append(total_faces[i][0])
    
num_faces = 6
face_size = latitudes.shape[0] // num_faces

all_lats = []
all_lons = []
all_vals = []

for i in range(num_faces):
    lat_face = latitudes[i * face_size:(i + 1) * face_size, :].values
    lon_face = longitudes[i * face_size:(i + 1) * face_size, :].values
    data_face = data_faces[i][0, :, :]  # shape: (y, x)

    lat_ds, lon_ds = downsample_latlon(lat_face, lon_face, data_face.shape)

    if lat_ds.shape != data_face.shape:
        print(f"Skipping face {i} due to shape mismatch: {lat_ds.shape} vs {data_face.shape}")
        continue

    all_lats.append(lat_ds.flatten())
    all_lons.append(lon_ds.flatten())
    all_vals.append(data_face.flatten())

merged_lat = np.concatenate(all_lats)
merged_lon = np.concatenate(all_lons)
merged_vals = np.concatenate(all_vals)

norm = mcolors.Normalize(vmin=np.min(merged_vals), vmax=np.max(merged_vals))
vmin = np.percentile(merged_vals, 2)
vmax = np.percentile(merged_vals, 98)

In [ ]:
%%time
def downsample_latlon(latitudes, longitudes, target_shape):
    factor_lat = latitudes.shape[0] // target_shape[0]
    factor_lon = latitudes.shape[1] // target_shape[1]
    latitudes_downsampled = latitudes[::factor_lat, ::factor_lon]
    longitudes_downsampled = longitudes[::factor_lat, ::factor_lon]
    return latitudes_downsampled, longitudes_downsampled

for t in tqdm(range(len(total_faces[0])), desc="Saving frames"):
    data_faces = []
    for i in range(len(total_faces)):
        data_faces.append(total_faces[i][t])

    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.set_global()

    num_faces = 6
    face_size = latitudes.shape[0] // num_faces

    all_lats = []
    all_lons = []
    all_vals = []

    for i in range(num_faces):
        lat_face = latitudes[i * face_size:(i + 1) * face_size, :].values
        lon_face = longitudes[i * face_size:(i + 1) * face_size, :].values
        data_face = data_faces[i][0, :, :]  # shape: (y, x)

        lat_ds, lon_ds = downsample_latlon(lat_face, lon_face, data_face.shape)

        if lat_ds.shape != data_face.shape:
            print(f"Skipping face {i} due to shape mismatch: {lat_ds.shape} vs {data_face.shape}")
            continue

        all_lats.append(lat_ds.flatten())
        all_lons.append(lon_ds.flatten())
        all_vals.append(data_face.flatten())

    merged_lat = np.concatenate(all_lats)
    merged_lon = np.concatenate(all_lons)
    merged_vals = np.concatenate(all_vals)

    scatter = ax.scatter(
        merged_lon,
        merged_lat,
        c=merged_vals,
        cmap='cividis',
        s=1,
        alpha=0.5,
        transform=ccrs.PlateCarree(),
        vmin=vmin,
        vmax=vmax
    )  

    gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree())
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlines = False
    gl.ylines = False
    gl.xlabel_style = {'size': 10}
    gl.ylabel_style = {'size': 10}
    cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', pad=0.05)

    os.makedirs(f'{variable}_images', exist_ok=True)

    plt.savefig(f'{variable}_images/frame_{t:03d}.png')
    plt.close(fig)
    

In [13]:
variables = ['T', 'W','U','V', 'QL', 'FCLD', 'DELP']

for variable in variables:
    os.makedirs(f'{variable}_images', exist_ok=True)
    data_file=[f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_0_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
              f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_1_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
              f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_2_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
              f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_3_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
              f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_4_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",
              f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS/GEOS_{variable.upper()}/{variable.lower()}_face_5_depth_52_time_0_10269.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco",]

    total_faces = []
    for actual_file_path in tqdm(data_file, desc="Loading faces"):
        db = ov.LoadDataset(actual_file_path)
        data=[]
        for i in tqdm(range(0, len(db.getTimesteps()), 24), desc="Weekly timesteps", leave=False):
            data.append(db.read(time=i,quality=-3,z=[50,51]))
        total_faces.append(data)

    data_faces = []
    for i in range(len(total_faces)):
        data_faces.append(total_faces[i][0])

    num_faces = 6
    face_size = latitudes.shape[0] // num_faces

    all_lats = []
    all_lons = []
    all_vals = []

    for i in range(num_faces):
        lat_face = latitudes[i * face_size:(i + 1) * face_size, :].values
        lon_face = longitudes[i * face_size:(i + 1) * face_size, :].values
        data_face = data_faces[i][0, :, :]  # shape: (y, x)

        lat_ds, lon_ds = downsample_latlon(lat_face, lon_face, data_face.shape)

        if lat_ds.shape != data_face.shape:
            print(f"Skipping face {i} due to shape mismatch: {lat_ds.shape} vs {data_face.shape}")
            continue

        all_lats.append(lat_ds.flatten())
        all_lons.append(lon_ds.flatten())
        all_vals.append(data_face.flatten())

    merged_lat = np.concatenate(all_lats)
    merged_lon = np.concatenate(all_lons)
    merged_vals = np.concatenate(all_vals)

    norm = mcolors.Normalize(vmin=np.min(merged_vals), vmax=np.max(merged_vals))
    vmin = np.percentile(merged_vals, 2)
    vmax = np.percentile(merged_vals, 98)

    for t in tqdm(range(len(total_faces[0])), desc="Saving frames"):
        data_faces = []
        for i in range(len(total_faces)):
            data_faces.append(total_faces[i][t])

        fig = plt.figure(figsize=(10, 8))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.coastlines()
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.set_global()

        num_faces = 6
        face_size = latitudes.shape[0] // num_faces

        all_lats = []
        all_lons = []
        all_vals = []

        for i in range(num_faces):
            lat_face = latitudes[i * face_size:(i + 1) * face_size, :].values
            lon_face = longitudes[i * face_size:(i + 1) * face_size, :].values
            data_face = data_faces[i][0, :, :]  # shape: (y, x)

            lat_ds, lon_ds = downsample_latlon(lat_face, lon_face, data_face.shape)

            if lat_ds.shape != data_face.shape:
                print(f"Skipping face {i} due to shape mismatch: {lat_ds.shape} vs {data_face.shape}")
                continue

            all_lats.append(lat_ds.flatten())
            all_lons.append(lon_ds.flatten())
            all_vals.append(data_face.flatten())

        merged_lat = np.concatenate(all_lats)
        merged_lon = np.concatenate(all_lons)
        merged_vals = np.concatenate(all_vals)

        scatter = ax.scatter(
            merged_lon,
            merged_lat,
            c=merged_vals,
            cmap='cividis',
            s=1,
            alpha=0.5,
            transform=ccrs.PlateCarree(),
            vmin=vmin,
            vmax=vmax
        )  

        gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree())
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xlines = False
        gl.ylines = False
        gl.xlabel_style = {'size': 10}
        gl.ylabel_style = {'size': 10}
        cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', pad=0.05)

        plt.savefig(f'{variable}_images/frame_{t:03d}.png', dpi=75)
        plt.close(fig)

Loading faces:   0%|          | 0/6 [00:00<?, ?it/s]

Weekly timesteps:   0%|          | 0/428 [00:00<?, ?it/s]

Weekly timesteps:   0%|          | 0/428 [00:00<?, ?it/s]

Weekly timesteps:   0%|          | 0/428 [00:00<?, ?it/s]

Weekly timesteps:   0%|          | 0/428 [00:00<?, ?it/s]

Weekly timesteps:   0%|          | 0/428 [00:00<?, ?it/s]

Weekly timesteps:   0%|          | 0/428 [00:00<?, ?it/s]

Saving frames:   0%|          | 0/428 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
variables = ['T']

for variable in variables:
    os.makedirs(f'{variable}_images', exist_ok=True)
    
    base_url = "https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/GEOS"
    base_params = "?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco"
    data_file = [
        f"{base_url}/GEOS_{variable.upper()}/{variable.lower()}_face_{i}_depth_52_time_0_10269.idx{base_params}"
        for i in range(6)
    ]

    total_faces = []
    for actual_file_path in tqdm(data_file, desc=f"Loading faces ({variable})"):
        db = ov.LoadDataset(actual_file_path)
        data = [db.read(time=i, quality=-4, z=[24, 25]) for i in range(0, len(db.getTimesteps()), 24)]
        total_faces.append(data)

    data_faces = [total_faces[i][0] for i in range(6)]

    num_faces = 6
    face_size = latitudes.shape[0] // num_faces

    all_lats, all_lons, all_vals = [], [], []

    for i in range(num_faces):
        lat_face = latitudes[i * face_size:(i + 1) * face_size, :].values
        lon_face = longitudes[i * face_size:(i + 1) * face_size, :].values
        data_face = data_faces[i][0, :, :]

        lat_ds, lon_ds = downsample_latlon(lat_face, lon_face, data_face.shape)

        if lat_ds.shape != data_face.shape:
            print(f"Skipping face {i} due to shape mismatch: {lat_ds.shape} vs {data_face.shape}")
            continue

        all_lats.append(lat_ds.flatten())
        all_lons.append(lon_ds.flatten())
        all_vals.append(data_face.flatten())

    merged_lat = np.concatenate(all_lats)
    merged_lon = np.concatenate(all_lons)
    merged_vals = np.concatenate(all_vals)

    vmin = np.percentile(merged_vals, 2)
    vmax = np.percentile(merged_vals, 98)

    for t in tqdm(range(len(total_faces[0])), desc=f"Saving frames ({variable})"):
        data_faces = [total_faces[i][t] for i in range(6)]

        fig = plt.figure(figsize=(10, 8))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.coastlines()
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.set_global()

        all_lats, all_lons, all_vals = [], [], []

        for i in range(num_faces):
            lat_face = latitudes[i * face_size:(i + 1) * face_size, :].values
            lon_face = longitudes[i * face_size:(i + 1) * face_size, :].values
            data_face = data_faces[i][0, :, :]

            lat_ds, lon_ds = downsample_latlon(lat_face, lon_face, data_face.shape)

            if lat_ds.shape != data_face.shape:
                print(f"Skipping face {i} due to shape mismatch: {lat_ds.shape} vs {data_face.shape}")
                continue

            all_lats.append(lat_ds.flatten())
            all_lons.append(lon_ds.flatten())
            all_vals.append(data_face.flatten())

        merged_lat = np.concatenate(all_lats)
        merged_lon = np.concatenate(all_lons)
        merged_vals = np.concatenate(all_vals)

        scatter = ax.scatter(
            merged_lon, merged_lat,
            c=merged_vals,
            cmap='cividis',
            s=1,
            alpha=0.5,
            transform=ccrs.PlateCarree(),
            vmin=vmin, vmax=vmax
        )

        gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree())
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xlines = False
        gl.ylines = False
        gl.xlabel_style = {'size': 10}
        gl.ylabel_style = {'size': 10}

        plt.colorbar(scatter, ax=ax, orientation='horizontal', pad=0.05)
        plt.savefig(f'{variable}_images/frame_{t:03d}.png', dpi=80)
        plt.close(fig)

Loading faces (T):   0%|          | 0/6 [00:00<?, ?it/s]

Saving frames (T):   0%|          | 0/428 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
 'W', 'U', 'V', 'QL', 'FCLD', 'DELP'